<a href="https://colab.research.google.com/github/TheAcoff/chat-Fismat/blob/main/Chat_Fismat_Obsidian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ferramenta de Auxílio em Estudos

Para este código, iremos utilizar dois modelos diferentes. Um para a detecção de intenção do usuário, tendo como saída variáveis precisas que irão entrar em uma lista para serem input de um segundo modelo que terá os dados treinados para a Física Matemática.

Para o modelo1, temos a saída:
$$
output1 = \begin{pmatrix} intenção \\ tema \\ ordem \end{pmatrix}  = input2
$$




In [1]:
!pip install PyPDF2

import google.generativeai as genai
import hashlib
import os
from google.colab import drive,userdata
from pathlib import Path
from typing import List
from PyPDF2 import PdfReader

#Exportando arquivos do Google Drive
drive.mount('/content/drive')
pasta_origem = Path('/content/drive/MyDrive/NotasObsidian')
pasta_nova = Path('/content/drive/MyDrive/NotasObsidian/Novas')

#Configuração de API
APIKey = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=APIKey)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Funções principais
Nessa parte, deixo todas as funções que estão sendo utilizadas para a manipulação de dados. Começando com uma função para converter a saída de texto do Gemini por uma matriz.

In [2]:
def string_para_lista(texto):
  # Remove as chaves e divide a string em elementos usando a vírgula como separador
  elementos = texto.strip("[]").split(",")

  elementos = [elemento for elemento in elementos]

  return elementos

Funções destinadas a extrair os textos dos arquivos:

In [3]:
def extract_pdf_pages(pathname: str) -> List[str]:
  parts = [f"--- Começo do arquivo {pathname} ---"]
  reader = PdfReader(pathname)
  pages = []

  for page in reader.pages:
    text = page.extract_text()
    pages.append(text)

  for index, page in enumerate(pages):
    parts.append(f"--- PAGE {index} ---")
    parts.append(page)

  return parts

In [4]:
def extract_md(pathname: str) -> list[str]:
  with open(f'{pathname}', 'r') as f:
      conteudo = f.read()
  return conteudo

Função destinada a importar todos os arquivos do Google Drive da sua pasta para o histórico de conversa do Gemini.

In [5]:
arquivos = []

# Looping pelos arquivos na pasta
for filename in os.listdir(pasta_origem):
    # Verifique se o arquivo é um .md
    if filename.endswith(".md"):
        # Crie o caminho completo para o arquivo
        filepath = os.path.join(pasta_origem, filename)

        # Adicione um dicionário à lista com as informações do arquivo
        arquivos.append({
            "role": "user",
            "parts": extract_md(filepath)
        })
    if filename.endswith(".pdf"):
        # Crie o caminho completo para o arquivo
        filepath = os.path.join(pasta_origem, filename)

        # Adicione um dicionário à lista com as informações do arquivo
        arquivos.append({
            "role": "user",
            "parts": extract_pdf_pages(filepath)
        })


Em alguns casos, as análises podem retornar nehuma intenção do usuário em uma ordem específica, para isso, limpamos a entrada para a sintaxe ser apenas um espaço no prompt:

In [6]:
def variavel_vazia(variavel, entrada):
  if variavel[entrada] == 'Nenhuma':
    saida = ' '
  else:
    saida = variavel[entrada]
  return saida

Essa parte do texto faz uma análise do usuário, gerando um feedback positivo, negativo ou de saída. O usuário pode querer resumir mais o texto, detalhar ou até mesmo gerar detalhes mais específicos:

In [7]:
def analise_usuario(prompt):
  #Modeo vai analisar a explicação do usuário e gerar novas variáveis
  output2 = chat2.send_message(prompt)
  output2M = string_para_lista(output2.text)

  feedback = output2M[0]
  ordem = variavel_vazia(output2M,1)
  detalhe = variavel_vazia(output2M,2)

  return feedback, ordem, detalhe


## Geração de Arquivos

In [8]:
def criar_markdown(conteudo, pasta, nome_arquivo="documento.md"):
  # Verificar se o nome do arquivo termina com ".md"
  if not nome_arquivo.endswith(".md"):
    nome_arquivo += ".md"

  # Criar a pasta se ela não existir
  os.makedirs(pasta, exist_ok=True)

  # Caminho completo para o arquivo
  caminho_arquivo = os.path.join(pasta, nome_arquivo)

  with open(caminho_arquivo, "w", encoding="utf-8") as f:
    f.write(conteudo)

  print(f"Arquivo '{caminho_arquivo}' criado com sucesso!")

### Geração de Texto

Para formatar a saída, facilitando a leitura do usuário, podemos:

In [9]:
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def imprimir_mensagem(output):
  display(to_markdown(f'{output.text}'))
  print('-------------------------------------------')

Separei dois prompts diferentes, um para criar texto novo, outro para editar um arquivo já existente.

In [10]:
def geracao_texto_novo(detalhe, tema, ordem):
  #Modelo vai gerar uma explicação para o usuário
  output1 = chat1.send_message(f'Baseado nos textos .md e no livro, explique {ordem} {detalhe} sobre {tema}, mantendo o formato dos arquivos .md')

  imprimir_mensagem(output1)

  return output1.text

In [11]:
def geracao_texto_edit(detalhe, tema, ordem,arquivo):
    output1 = chat1.send_message(f'Sugira modificações para o {arquivo}.md sobre {tema}. \n Durante o texto, faça os links dos termos mais importantes com [[]] como é feito nos arquivos enviados.  Escreva um arquivo em .md. Coloque equações em usando $. Explique {ordem} {detalhe}')

    imprimir_mensagem(output1)

    return output1.text

# Definição de Parâmetros
Nessa parte, definimos os parâmetros dos dois modelos. Seja $n$ o número do modelo, podendo ser $0$ ou $1$ para os parâmetros, então, definimos previamente: generation_config$\,n$ , safety_settings$\,n$

## Parâmetros do Modelo 0 e 2

In [12]:
generation_config0 = {
  "temperature": 0.3,
  "top_p": 0,
  "top_k": 0,
  "max_output_tokens": 4096
}

safety_settings0 = {
    'HARASSMENT':'BLOCK_MEDIUM_AND_ABOVE',
    'HATE':'BLOCK_MEDIUM_AND_ABOVE',
    'SEXUAL':'BLOCK_MEDIUM_AND_ABOVE',
    'DANGEROUS':'BLOCK_MEDIUM_AND_ABOVE',
}

## Parâmetros do Modelo 1

In [13]:
generation_config1 = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 12000
}

system_instruction1 = "Aja como um auxiliar para anotações no obsidian. Desconsidere o título, faça os headings com '#' e os subheadings com '##'"

safety_settings1 = {
    'HARASSMENT':'BLOCK_NONE',
    'HATE':'BLOCK_NONE',
    'SEXUAL':'BLOCK_NONE',
    'DANGEROUS':'BLOCK_NONE',
}

# Incializando os Modelos

In [14]:
# Modelo para primeira avaliação do input do usuário
model0 = genai.GenerativeModel(
    model_name='gemini-1.0-pro',
    generation_config = generation_config0,
    safety_settings = safety_settings0,
    )

#Modelo principal de geração de textos
model1 = genai.GenerativeModel(
    model_name='gemini-1.5-pro-latest',
    generation_config = generation_config1,
    safety_settings = safety_settings1,
    system_instruction = system_instruction1,
    )

#Modelo para validar a opinião do usuário com base no resultado do 'model1'
model2 = genai.GenerativeModel(
    model_name='gemini-1.0-pro',
    generation_config = generation_config0,
    safety_settings = safety_settings0,
    )

In [15]:
chat0 = model0.start_chat(history=[
  {
    "role": "user",
    "parts": ["Aja como um identificador de intenção e ordem do usuário e me ajude a detectar o tema proposto. \nDevemos ter dois resultados possíveis de Intenção desejadas pelo usuário: Dúvida e Edição. \nJá para ordem: Nenhuma, Resuma, Detalhe, Crie.\nPara isso, responda no seguinte formato, como se fosse declarar uma matriz em Python, sem utilizar os espaços: [Intenção,Ordem,Tema] \n---\nExemplo1: Gostaria de saber mais sobre Inteligência Artificial \n[Dúvida,Detalhe,Inteligência Artificial]\nExemplo2: Me ajude com minhas notas sobre Redes Neurais \n[Edição,Complemente,Redes Neurais] \nExemplo3: Crie uma nota sobre Produto Interno\n[Geração,Crie,Produto Interno]\nExemplo4: Me explique sobre Espaço Vetorial. \n[Dúvida,Nenhma,Espaço Vetorial] \nExemplo 5: Fale mais sobre Jupiter e suas luas \n[Dúvida,Detalhe,Luas de Júpiter] \nExemplo 6: Resume sobre crateras da Terra \n[Dúvida,Resuma,Crateras da Terra] \nExemplo7: Resuma minhas notas sobre Teclado \n[Edição,Resuma,Teclado]"]
  },
  {
    "role": "model",
    "parts": ["Ok, irei analisar as frases e fazer no modelo [Intenção,Ordem,Tema]"]
  }
])

chat1 = model1.start_chat(history = arquivos)

chat2 = model2.start_chat(history=[
  {
    "role": "user",
    "parts": ["Aja como um identificador de aprovação do usuário. Me ajude a detectar a categoria, a ordem e o detalhe da frase do usuário. \n\nDevemos ter três tipos de categoria: aprovado, desaprovado e sair.\n\nDevemos ter três tipos de ordem: resuma, detalhe, nenhuma\n\nPara o detalhe, analise qual parte o usuário deseja uma explicação, caso categoria for desaprovado. Se for diferente de desaprovado, a resposta será nenhuma.\n\nO formato de saída deve ser como uma lista em Python: [categoria,ordem,detalhe]\n\n---\nExemplo1:  Não estou satisfeito com a resposta.\n[Desaprovado,Nenhuma,Nenhuma]\n\nExemplo2: Não gostei, resuma mais um pouco.\n[Desaprovado,Resuma,Nenhum]\n\nExemplo3: Resuma mais a parte do terceiro parágrafo do assunto\n[Desaprovou, Resuma, terceiro parágrafo do assunto]\n\nExemplo4: Obrigado, estou satisfeito\n[Aprovado,Nenhuma,Nenhuma]\n\nExemplo5: Detalhe mais sobre o capitalismo\n[Desaprovado,Detalhe,capitalismo]\n\nExemplo6: Não precisa gerar mais\n[Sair,Nenhuma,Nenhuma]\n\nExemplo7: Tudo bem, eu pergunto outra coisa\n[Sair,Nenhuma,Nenhuma]\n\nExemplo8: Explique melhor as equações\n[Desaprovado,Detalhe,equações]"]
  },
  {
    "role": "model",
    "parts": ["[Desaprovado, Nenhuma, Nenhuma]\n[Desaprovado, Resuma, Nenhuma]\n[Desaprovado, Resuma, terceiro parágrafo do assunto]\n[Aprovado, Nenhuma, Nenhuma]\n[Desaprovado, Detalhe, capitalismo]\n[Sair, Nenhuma, Nenhuma]\n[Sair, Nenhuma, Nenhuma]\n[Desaprovado, Detalhe, equações]"]
  },
  {
    "role": "user",
    "parts": ["Obrigado, não quero mais informações"]
  },
  {
    "role": "model",
    "parts": ["[Sair, Nenhuma, Nenhuma]"]
  },
])

# Código em Ação

O código irá pegar uma primeira entrada do usuário para gerar um texto de explicação ou uma sugestão de edição nas notas. Caso o usuário aceite as alterações, ele irá jogar para uma nova pasta!

In [16]:
#Declaração de Variáveis
prompt = str(input('O que deseja saber mais? (digite: "fim" para encerrar): '))
detalhe = str(' ')
ordem = str(' ')

while prompt != 'fim':
  #Envia o prompt e converte a saída para uma matriz.
  output0 = chat0.send_message(prompt)
  output0M = string_para_lista(output0.text)

  #Separa as variáveis da saída
  intencao = output0M[0]
  tema = output0M[2]
  ordem = variavel_vazia(output0M,1)

  #Verifica se a intenção é Dúvida ou Edição
  feedback = str(' ')

  if intencao == 'Dúvida':
    #Rodamos o código enquanto o usuário não estiver satisfeito:
    while (feedback != 'Aprovado' and feedback != 'Sair'):
      #Geramos os textos para o usuário:
      output1 = geracao_texto_novo(detalhe, tema, ordem)

      #Coletando resposta
      prompt = str(input('Gostou do texto gerado?: '))
      feedback, ordem, detalhe = analise_usuario(prompt)



  else:
    #Rodamos o código enquanto o usuário não estiver satisfeito:
    while (feedback != 'Aprovado' and feedback != 'Sair'):
      #Geramos os textos para o usuário
      output1 = geracao_texto_edit(detalhe,tema,ordem,tema)

      #Coletando resposta
      prompt = str(input('Gostou do texto gerado?: '))
      feedback, ordem, detalhe = analise_usuario(prompt)

      if (feedback == 'Aprovado'):
        #Criar novo arquivo .md com as notas geradas (output1) e nome Tema
        criar_markdown(output1,pasta_nova,tema)

  prompt = str(input('O que deseja saber mais?: '))




O que deseja saber mais? (digite: "fim" para encerrar): Faça uma nota sobre Grupo SO(3)


> # Grupo SO(3)
> 
> O grupo SO(3) (Grupo Ortogonal Especial em 3 dimensões) é o grupo de todas as [[Matriz de Rotação|matrizes de rotação]] em 3 dimensões. As matrizes de rotação são matrizes ortogonais com determinante igual a +1.  Elas representam rotações em torno de um eixo que passa pela origem.
> 
> ## Definição
> 
> O grupo SO(3) é definido como:
> 
> $$
> SO(3) = \{R \in \mathbb{R}^{3\times3} | R^TR = I, \det(R) = 1 \}
> $$
> 
> Onde:
> 
> * $R$ é uma matriz de rotação 3x3.
> * $R^T$ é a transposta da matriz $R$.
> * $I$ é a matriz identidade 3x3.
> * $\det(R)$ é o determinante da matriz $R$.
> 
> ## Propriedades 
> 
> O grupo SO(3) tem as seguintes propriedades:
> 
> * **Fechamento:** O produto de duas matrizes de rotação é uma matriz de rotação.
> * **Associatividade:** A multiplicação de matrizes é associativa.
> * **Elemento Neutro:** A matriz identidade $I$ é o elemento neutro do grupo.
> * **Elemento Inverso:** Toda matriz de rotação $R$ possui uma inversa $R^{-1}$, que também é uma matriz de rotação.
> 
> ## Representações
> 
> Existem diferentes maneiras de representar o grupo SO(3). 
> 
> * **Matrizes 3x3:** Esta é a representação mais comum. Uma matriz de rotação 3x3 pode ser parametrizada por três ângulos, como os ângulos de Euler.
> * **Quaternions:** Os quaternions são uma extensão dos números complexos que podem ser usados para representar rotações de forma mais compacta e eficiente.
> * **Ângulos de Euler:** Os ângulos de Euler (α, β, γ)  são um conjunto de três ângulos que parametrizam uma rotação em três dimensões. 
> 
> ## Aplicações
> 
> O grupo SO(3) tem várias aplicações em física, matemática e engenharia, incluindo:
> 
> * **Mecânica Clássica:**  Descrevendo a orientação de corpos rígidos.
> * **Robótica:**  Controlando a orientação de robôs.
> * **Computação Gráfica:**  Realizando rotações de objetos em 3D.
> * **Teoria Quântica:**  Representando o spin de partículas elementares.
> 
> ## Conexão com outros grupos
> 
> O grupo SO(3) está intimamente relacionado com o [[Grupo SU(2)|grupo SU(2)]], que é o grupo de todas as matrizes unitárias 2x2 com determinante igual a +1.  Existe um homomorfismo 2:1 de SU(2) em SO(3).
> 
> ## Conclusão
> 
> O grupo SO(3) é um grupo fundamental com aplicações importantes em diversas áreas. A compreensão de suas propriedades e representações é essencial para o estudo de rotações em três dimensões. 


-------------------------------------------
Gostou do texto gerado?: Sim, obrigado
Arquivo '/content/drive/MyDrive/NotasObsidian/Novas/Grupo SO(3).md' criado com sucesso!
O que deseja saber mais?: fim
